In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [3]:
root_dir = Path().cwd().parent
interim_dir = root_dir / 'data/interim'
undistorted_dir = interim_dir / 'undistorted'
rectified_dir = interim_dir / 'rectified'
rectified_dir.mkdir(parents=True, exist_ok=True)

In [4]:
folders = os.listdir(undistorted_dir)
for folder in folders:
    images = os.listdir(undistorted_dir / folder)